In [ ]:
#we would like to create a doggy door that only lets in a particular dog
#we will make an automatic doggy door for a dog named Bo (also called the presidential dog)

#downloading the pre-trained model
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  
    input_shape=(224, 224, 3),
    include_top=False)

#The last layer of an ImageNet model is a dense layer of 1000 units, representing the 1000 possible classes in the dataset. 
#In our case, we want it to make a different classification: is this Bo or not? 
#Because we want the classification to be different, we are going to remove the last layer of the model. 
base_model.summary()

#Freezing the Base Model
base_model.trainable = False

#Adding New Layers
inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to False 
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

#Compiling the Model
#we have a binary classification problem (Bo or not Bo)
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])
#from_logits=True we inform the loss function that the output values are not normalized 

#Augmenting the Data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

#Loading the Data
# we are going to load images directly from folders using Keras' flow_from_directory function
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/presidential_doggy_door/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/presidential_doggy_door/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='binary', 
                                      batch_size=8)

#Training the Model
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)







#Fine-Tuning the Model

# Unfreeze the base model
base_model.trainable = True

# recompilation
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])


model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)





#Examining the Predictions
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(224, 224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

#make_predictions('data/presidential_doggy_door/valid/bo/bo_20.jpg')

#make_predictions('data/presidential_doggy_door/valid/not_bo/121.jpg')

def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if preds[0]<0:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")
        
#presidential_doggy_door('data/presidential_doggy_door/valid/not_bo/131.jpg')

presidential_doggy_door('data/presidential_doggy_door/valid/bo/bo_29.jpg')
       